# Programmatic Tool Calling (Experimental)

Emulate Anthropic's Programmatic Tool Calling pattern in LangChain.

Ref:

- https://platform.claude.com/docs/agents-and-tools/tool-use/programmatic-tool-calling

**Problem**: Traditional tool calling requires API round-trips for each tool call.
With many tool calls, this adds latency and token overhead.

**Solution**: Let the agent generate Python code that calls multiple tools,
execute the code in a sandbox, and return only the final result.

**Benefits**:

- Reduced latency (batch multiple tool calls)
- Token efficiency (aggregate results before returning to model)
- Complex workflows (loops, conditionals over tool results)

Uses [sandbox-runtime](https://github.com/anthropic-experimental/sandbox-runtime) for secure execution.
No global install required - runs via `npx`.


## Setup


In [43]:
import json
import re
import shutil
import tempfile
from pathlib import Path
from typing import Any

import rich
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import BaseTool, tool
from langgraph.graph.state import CompiledStateGraph

load_dotenv()

# Check for srt - prefer global install, fallback to npx
SRT_CMD: list[str]
if shutil.which("srt"):
    SRT_CMD = ["srt"]
    rich.print("[green]srt found (global)[/green]")
elif shutil.which("npx"):
    SRT_CMD = ["npx", "@anthropic-ai/sandbox-runtime"]
    rich.print("[green]srt available via npx[/green]")
else:
    raise RuntimeError("srt not available. Install Node.js for npx support.")

srt available via npx

## Define Tool Registry

These tools can be called programmatically from generated code via `tool_call()`.


In [44]:
@tool
def query_sales(region: str) -> dict[str, Any]:
    """Query sales data for a region.

    Returns: {"region": str, "revenue": int, "units": int, "top_product": str}
    Error: {"error": str} if region not found.
    """
    data = {
        "west": {"revenue": 150000, "units": 1200, "top_product": "Widget A"},
        "east": {"revenue": 220000, "units": 1800, "top_product": "Widget B"},
        "central": {"revenue": 180000, "units": 1500, "top_product": "Widget A"},
        "north": {"revenue": 95000, "units": 800, "top_product": "Widget C"},
        "south": {"revenue": 130000, "units": 1100, "top_product": "Widget B"},
    }
    region_lower = region.lower()
    if region_lower in data:
        return {"region": region, **data[region_lower]}
    return {"error": f"Unknown region: {region}"}


@tool
def get_weather(city: str) -> dict[str, Any]:
    """Get current weather for a city.

    Returns: {"city": str, "temp": int, "condition": str, "humidity": int}
    Error: {"error": str} if city not found.
    """
    data = {
        "tokyo": {"temp": 22, "condition": "Sunny", "humidity": 45},
        "new york": {"temp": 18, "condition": "Cloudy", "humidity": 60},
        "london": {"temp": 15, "condition": "Rainy", "humidity": 80},
        "paris": {"temp": 20, "condition": "Partly Cloudy", "humidity": 55},
        "sydney": {"temp": 25, "condition": "Sunny", "humidity": 40},
    }
    city_lower = city.lower()
    if city_lower in data:
        return {"city": city, **data[city_lower]}
    return {"error": f"Unknown city: {city}"}


TOOL_REGISTRY: dict[str, BaseTool] = {t.name: t for t in [query_sales, get_weather]}

rich.print("Available tools:", list(TOOL_REGISTRY.keys()))

Available tools:
['query_sales', 'get_weather']

## Tool Search

Allows agent to discover available tools by pattern.


In [45]:
@tool
def tool_search(pattern: str) -> list[dict[str, Any]]:
    """Search for available tools by regex pattern.

    Args:
        pattern: Regex pattern to match against tool names and descriptions.

    Returns: list of tool schemas with properties, required, description, etc.
    Empty list if no tools found.
    """
    rich.print(f"[bold cyan]tool_search({pattern!r})[/bold cyan]")
    matches = []
    for name, t in TOOL_REGISTRY.items():
        if re.search(pattern, name, re.IGNORECASE) or re.search(pattern, t.description, re.IGNORECASE):
            if t.args_schema and hasattr(t.args_schema, "model_json_schema"):
                schema = t.args_schema.model_json_schema()
                schema["name"] = name  # Add tool name
                matches.append(schema)
    rich.print(f"[dim]{json.dumps(matches, indent=2)}[/dim]")
    return matches

## Sandboxed Code Execution Tool

Execute Python code in a sandbox with `tool_call()` to invoke registered tools.
Uses JSON-RPC over stdin/stdout to communicate with the host.


In [46]:
import asyncio


def create_sandboxed_execute_code_tool(registry: dict[str, BaseTool]) -> BaseTool:
    """Create a sandboxed code execution tool using srt (async version)."""

    async def execute_code_impl(code: str) -> str:
        """Execute async Python code in a sandbox.

        Available functions:
        - await tool_call(name, **kwargs): Call a registered tool, returns dict
        - print(): Output results (only printed text is returned)

        Example:
            sales = await tool_call("query_sales", region="west")
            print(f"Revenue: {sales['revenue']}")
        """
        rich.print("[bold cyan]Executing sandboxed code:[/bold cyan]")
        rich.print(code)
        rich.print("[bold cyan]---[/bold cyan]")

        # Indent user code for async main()
        indented_code = "\n".join("    " + line for line in code.split("\n"))

        wrapper = f'''
import asyncio
import json
import sys

async def tool_call(name, **kwargs):
    """Call a tool on the host via stdout/stdin protocol."""
    request = {{"name": name, "kwargs": kwargs}}
    sys.stdout.write(f"__TOOL_REQUEST__{{json.dumps(request)}}__END_REQUEST__\\n")
    sys.stdout.flush()
    response_line = sys.stdin.readline().strip()
    return json.loads(response_line)

_output_lines = []
_original_print = print
def print(*args, **kwargs):
    import io
    buf = io.StringIO()
    kwargs["file"] = buf
    _original_print(*args, **kwargs)
    _output_lines.append(buf.getvalue())

async def main():
{indented_code}

asyncio.run(main())

sys.stdout.write("__USER_OUTPUT__\\n")
sys.stdout.flush()
for line in _output_lines:
    sys.stdout.write(line)
sys.stdout.flush()
'''

        with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as f:
            f.write(wrapper)
            script_path = f.name

        try:
            proc = await asyncio.create_subprocess_exec(
                *SRT_CMD,
                "python",
                "-u",
                script_path,
                stdin=asyncio.subprocess.PIPE,
                stdout=asyncio.subprocess.PIPE,
                stderr=asyncio.subprocess.PIPE,
            )
            assert proc.stdin is not None
            assert proc.stdout is not None
            assert proc.stderr is not None
            proc_stdin = proc.stdin
            proc_stdout = proc.stdout
            proc_stderr = proc.stderr

            output_lines: list[str] = []
            user_output_started = False

            while True:
                try:
                    line_bytes = await asyncio.wait_for(proc_stdout.readline(), timeout=30)
                except TimeoutError:
                    break

                if not line_bytes:
                    break

                line = line_bytes.decode()

                if "__TOOL_REQUEST__" in line:
                    start = line.index("__TOOL_REQUEST__") + len("__TOOL_REQUEST__")
                    end = line.index("__END_REQUEST__")
                    request = json.loads(line[start:end])

                    tool_name = request["name"]
                    tool_kwargs = request["kwargs"]

                    if tool_name in registry:
                        tool_obj = registry[tool_name]
                        # Use ainvoke for async tools (like MCP tools)
                        if hasattr(tool_obj, "coroutine") and tool_obj.coroutine is not None:
                            call_result = await tool_obj.ainvoke(tool_kwargs)
                        else:
                            call_result = tool_obj.invoke(tool_kwargs)
                        rich.print(f"[dim]  await tool_call({tool_name!r}, {tool_kwargs}) -> {call_result}[/dim]")
                    else:
                        call_result = {"error": f"Unknown tool: {tool_name}"}

                    proc_stdin.write((json.dumps(call_result) + "\n").encode())
                    await proc_stdin.drain()
                elif "__USER_OUTPUT__" in line:
                    user_output_started = True
                elif user_output_started:
                    output_lines.append(line)

            stderr_bytes = await proc_stderr.read()
            stderr = stderr_bytes.decode()
            await proc.wait()

            output = "".join(output_lines)
            if stderr:
                output += f"\nStderr: {stderr}"
            if proc.returncode != 0:
                output += f"\nExit code: {proc.returncode}"

            return output.strip() if output.strip() else "Code executed successfully (no output)"

        except TimeoutError:
            proc.kill()
            return "Error: Execution timed out (30s limit)"
        except Exception as e:
            return f"Error: {type(e).__name__}: {e}"
        finally:
            Path(script_path).unlink(missing_ok=True)

    # Create StructuredTool with async coroutine
    from langchain_core.tools import StructuredTool

    return StructuredTool.from_function(
        coroutine=execute_code_impl,
        name="execute_code",
        description="""Execute async Python code in a sandbox.

Available functions:
- await tool_call(name, **kwargs): Call a registered tool, returns dict
- print(): Output results (only printed text is returned)

Example:
    sales = await tool_call("query_sales", region="west")
    print(f"Revenue: {sales['revenue']}")""",
    )


execute_code_tool = create_sandboxed_execute_code_tool(TOOL_REGISTRY)
rich.print("Sandboxed execute_code tool created (async)")

Sandboxed execute_code tool created (async)

## Create Agent


In [47]:
model = ChatAnthropic(model="claude-sonnet-4-5-20250929")

SYSTEM_PROMPT = """You are a data analyst assistant.

You have access to:
- tool_search: Search for available tools by regex pattern
- execute_code: Run Python code in a secure sandbox with tool_call(name, **kwargs)

Workflow:
1. Use tool_search to find relevant tools
2. Use execute_code with tool_call() to invoke them in batch
3. Aggregate results and print the final summary"""

agent: CompiledStateGraph[Any] = create_agent(
    model=model,
    tools=[tool_search, execute_code_tool],
    system_prompt=SYSTEM_PROMPT,
)

rich.print("Agent created")

Agent created

## Test: Multi-Region Analysis


In [48]:
query = """For each region, get sales data and check weather in its capital city.
Capital cities: West=Sydney, East=New York, Central=London, North=Tokyo, South=Paris.

Calculate revenue per unit for each region, then recommend the best region to visit
considering both business performance (high revenue per unit) and weather (sunny preferred)."""

result = await agent.ainvoke({"messages": [{"role": "user", "content": query}]})

rich.print("\n[bold green]Final response:[/bold green]")
rich.print(result["messages"][-1].content)

tool_search('sales|weather')

[
  {
    "description": "Query sales data for a region.\n\nReturns: {\"region\": str, \"revenue\": int, \"units\": int, 
\"top_product\": str}\nError: {\"error\": str} if region not found.",
    "properties": {
      "region": {
        "title": "Region",
        "type": "string"
      }
    },
    "required": [
      "region"
    ],
    "title": "query_sales",
    "type": "object",
    "name": "query_sales"
  },
  {
    "description": "Get current weather for a city.\n\nReturns: {\"city\": str, \"temp\": int, \"condition\": str, 
\"humidity\": int}\nError: {\"error\": str} if city not found.",
    "properties": {
      "city": {
        "title": "City",
        "type": "string"
      }
    },
    "required": [
      "city"
    ],
    "title": "get_weather",
    "type": "object",
    "name": "get_weather"
  }
]

Executing sandboxed code:

# Define regions and their capital cities
regions_capitals = {
    "West": "Sydney",
    "East": "New York",
    "Central": "London",
    "North": "Tokyo",
    "South": "Paris"
}

# Gather sales data for all regions
sales_data = {}
for region in regions_capitals.keys():
    result = await tool_call("query_sales", region=region)
    sales_data = result
    print(f"Sales data for {region}: {result}")

print("\n" + "="*60 + "\n")

# Gather weather data for all capital cities
weather_data = {}
for region, city in regions_capitals.items():
    result = await tool_call("get_weather", city=city)
    weather_data = result
    print(f"Weather in {city} ({region}): {result}")

---

...

Executing sandboxed code:

# Define regions and their capital cities
regions_capitals = {
    "West": "Sydney",
    "East": "New York",
    "Central": "London",
    "North": "Tokyo",
    "South": "Paris"
}

# Use the previously gathered data
sales_data = {
    'West': {'region': 'West', 'revenue': 150000, 'units': 1200, 'top_product': 'Widget A'},
    'East': {'region': 'East', 'revenue': 220000, 'units': 1800, 'top_product': 'Widget B'},
    'Central': {'region': 'Central', 'revenue': 180000, 'units': 1500, 'top_product': 'Widget A'},
    'North': {'region': 'North', 'revenue': 95000, 'units': 800, 'top_product': 'Widget C'},
    'South': {'region': 'South', 'revenue': 130000, 'units': 1100, 'top_product': 'Widget B'}
}

weather_data = {
    'West': {'city': 'Sydney', 'temp': 25, 'condition': 'Sunny', 'humidity': 40},
    'East': {'city': 'New York', 'temp': 18, 'condition': 'Cloudy', 'humidity': 60},
    'Central': {'city': 'London', 'temp': 15, 'condition': 'Rainy', 'humidity': 80},
    'North': {'city': 'Tokyo', 'temp': 22, 'condition': 'Sunny', 'humidity': 45},
    'South': {'city': 'Paris', 'temp': 20, 'condition': 'Partly Cloudy', 'humidity': 55}
}

# Calculate revenue per unit for each region
print("REVENUE PER UNIT ANALYSIS")
print("="*60)

revenue_per_unit = {}
for region in regions_capitals.keys():
    sales = sales_data
    rpu = sales['revenue'] / sales['units']
    revenue_per_unit = rpu
    print(f"{region:10} | Revenue: ${sales['revenue']:,} | Units: {sales['units']:,} | RPU: ${rpu:.2f}")

print("\n" + "="*60 + "\n")

# Rank regions by revenue per unit
print("BUSINESS PERFORMANCE RANKING (Revenue per Unit)")
print("="*60)
sorted_by_rpu = sorted(revenue_per_unit.items(), key=lambda x: x[1], reverse=True)
for i, (region, rpu) in enumerate(sorted_by_rpu, 1):
    print(f"{i}. {region:10} - ${rpu:.2f} per unit")

print("\n" + "="*60 + "\n")

# Weather analysis
print("WEATHER CONDITIONS")
print("="*60)
for region, city in regions_capitals.items():
    weather = weather_data
    print(f"{region:10} ({city:10}) | {weather['condition']:15} | {weather['temp']}°C | Humidity: 
{weather['humidity']}%")

print("\n" + "="*60 + "\n")

# Identify regions with sunny weather
sunny_regions = [region for region in regions_capitals.keys() 
                 if weather_data['condition'] == 'Sunny']
print(f"Regions with Sunny weather: {', '.join(sunny_regions)}")

print("\n" + "="*60 + "\n")

# Make recommendation
print("RECOMMENDATION")
print("="*60)

# Score each region (higher is better)
scores = {}
for region in regions_capitals.keys():
    # Business score (normalize RPU to 0-100 scale)
    rpu_rank = len(sorted_by_rpu) - .index(region)
    business_score = (rpu_rank / len(sorted_by_rpu)) * 100
    
    # Weather score
    weather = weather_data['condition']
    if weather == 'Sunny':
        weather_score = 100
    elif weather == 'Partly Cloudy':
        weather_score = 70
    elif weather == 'Cloudy':
        weather_score = 40
    else:  # Rainy
        weather_score = 10
    
    # Combined score (60% business, 40% weather)
    total_score = (business_score * 0.6) + (weather_score * 0.4)
    scores = {
        'business_score': business_score,
        'weather_score': weather_score,
        'total_score': total_score,
        'rpu': revenue_per_unit,
        'weather': weather
    }

# Sort by total score
ranked = sorted(scores.items(), key=lambda x: x[1]['total_score'], reverse=True)

for i, (region, data) in enumerate(ranked, 1):
    print(f"\n{i}. {region} (Capital: {regions_capitals})")
    print(f"   Revenue per Unit: ${data['rpu']:.2f}")
    print(f"   Weather: {data['weather']}")
    print(f"   Business Score: {data['business_score']:.1f}/100")
    print(f"   Weather Score: {data['weather_score']:.1f}/100")
    print(f"   TOTAL SCORE: {data['total_score']:.1f}/100")

print("\n" + "="*60)
best_region = ranked[0][0]
best_data = ranked[0][1]
print(f"\n🏆 BEST REGION TO VISIT: {best_region} (Capital: {regions_capitals})")
print(f"\n   ✓ Strong revenu

---

Final response:

## Summary

Based on the comprehensive analysis of sales data and weather conditions across all regions, here's my 
recommendation:

### **🏆 Best Region to Visit: WEST (Sydney)**

**Why West/Sydney is the top choice:**

1. **Best Business Performance**: Revenue per unit of **$125.00** - the highest among all regions
   - Total revenue: $150,000
   - Units sold: 1,200
   - Top product: Widget A

2. **Perfect Weather**: **Sunny** conditions with excellent visiting weather
   - Temperature: 25°C (77°F)
   - Low humidity: 40%
   - Ideal for outdoor business activities

3. **Perfect Score**: 100/100 overall - combining excellent business metrics (100/100) with ideal weather 
conditions (100/100)

**Honorable Mentions:**
- **East (New York)**: Second-best revenue per unit ($122.22) but cloudy weather
- **North (Tokyo)**: Sunny weather like Sydney but lower revenue per unit ($118.75)

The West region offers the best combination of strong business performance and favorable weather conditions, making
it the optimal choice for your visit.